# Everactive ENV+ EvalKit Exploration Notebook

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/everactive/python-notebook-sample/blob/first-draft/Env%2B_EvalKit_Exploration.ipynb)

This notebook is designed to give you a quickstart to accessing your [ENV+ EvalKit](https://everactive.com/product/environmental-evaluation-kit/) sensor data via the Everactive Data Services API and creating visualizations using your data.

## Before You Begin
* This notebook assumes that you are familiar with programming in Python, notebook operation and execution (e.g. JupyterLab or Google Colab), and the `pandas` library.

* We'll use the `altair` library for visualizations. Prior experience with `altair` is not required, but if you would like to learn more about the library and how it works, please visit [their excellent documentation](https://altair-viz.github.io/getting_started/overview.html).

* This notebook pulls *your* sensor data, which will be different from the data used to develop this notebook. Though we've endeavored to craft this notebook to accommodate wide range of returned sensor readings, it is possible that your data might contain an unforseen edge case. If that's the case, and your data breaks the notebook or visualizations, please create a GitHub issue.

If you're running this notebook in Google Colab, there's a few setup steps that we need to run first to prepare the Colab environment.

In [ ]:
# Check if we're running locally, or in Google Colab.
try:
    import google.colab
    print("Running in Google Colab, installing everactive_envplus library...")
    !pip install "git+https://github.com/everactive/python-notebook-sample.git@first-draft"
    print("Install complete.")

except ModuleNotFoundError:
    pass

## Notebook Imports

In [2]:
import altair as alt
import pandas as pd

import everactive_envplus as ee

The `everactive_envplus` library that we will use within this notebook provides a wrapper around the Everactive Data Services API. Its interface is designed to make pulling your Eversensor and Evergateway data easy. 

If you'd like to dive deeper into the Everactive Data Services API interface, please check out our [API Documentation](https://docs.api.everactive.com/reference/data-services-api-overview).

## Provide API Credentials

<mark>*To get started, provide your API credentials below:*</mark>

In [3]:
CLIENT_ID = ""
CLIENT_SECRET = ""

## Establish a Connection to the API

First, we'll use your API credentials to connect to the Everactive Data Services API.

In [4]:
conn = ee.connection.ApiConnection(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)

api = ee.EveractiveApi(api_connection=conn)

[DEBUG] 2023-01-23 16:34:24 (884) api_connection: Fetching oauth token from: https://api.data.everactive.com/auth/token
[INFO] 2023-01-23 16:34:24 (884) api_connection: Authenticated to the Everactive API


## Pull Eversensors Associated with Your Account

To get all of the Eversensors associated with your account, use the `get_all_eversensors()` call. All methods that return API data accept an `output_format` argument that specifies `json` (the default) or `pandas`.

In [5]:
eversensors = api.get_all_eversensors()
print(f"Fetched {len(eversensors)} eversensors.")

[DEBUG] 2023-01-23 16:34:25 (884) api_connection: Requested URL: https://api.data.everactive.com/ds/v1/eversensors?page=1&page-size=500&sort-by=mac-address&devkitBundled=True
[DEBUG] 2023-01-23 16:34:25 (884) api_connection: Page: 1/1, Total Items: 85


Fetched 85 eversensors.


In [6]:
eversensors[0]

{'customer': {'id': '5a47e64a-aa0a-43fd-b801-732247e8ae5e',
  'isSandbox': False,
  'name': 'NUS',
  'status': 'Production'},
 'devkitBundled': True,
 'lastAssociation': {'gatewaySerialNumber': '',
  'timestamp': '2022-12-06T20:52:17.000Z'},
 'lastInfo': {'firmwareVersion': 'tinytiger-f0/v1.0.0-r1232d30e',
  'gatewaySerialNumber': '',
  'partNumber': '810-00039',
  'sensorSerialNumber': '22420008',
  'timestamp': '2022-12-06T20:53:01.000Z'},
 'lastReading': {'scap': 1.75,
  'vcap': 2.4375,
  'schema': 'https://evr.ac/schemas/platform/eversensor/environmental/v1/reading',
  'timestamp': 1670339260,
  'loadCounts': [{'trim': 4, 'count': 15, 'measurementTime': 10}],
  'macAddress': 'bc:5e:a1:00:00:00:5e:17',
  'rssiUplink': -58,
  'readingDate': '2022-12-06T15:07:40Z',
  'unsolicited': False,
  'schemaVersion': 'v1',
  'gatewaySerialNumber': 'PMM-QA0002',
  'movementMeasurement': {'initialAcceleration': {'x': 48,
    'y': -60,
    'z': -1005}},
  'packetNumberGateway': 84197,
  'pressureM

## Pull Readings for a Single Eversensor

In [7]:
sensor_mac_address = eversensors[0]["macAddress"]
sensor_mac_address

'bc:5e:a1:00:00:00:5e:17'

### Examine Returned Eversensor Reading Schema

In [8]:
last_reading = api.get_eversensor_last_reading(sensor_mac_address)
last_reading

{'scap': 1.75,
 'vcap': 2.4375,
 'schema': 'https://evr.ac/schemas/platform/eversensor/environmental/v1/reading',
 'timestamp': 1670339260,
 'loadCounts': [{'trim': 4, 'count': 15, 'measurementTime': 10}],
 'macAddress': 'bc:5e:a1:00:00:00:5e:17',
 'rssiUplink': -58,
 'readingDate': '2022-12-06T15:07:40Z',
 'unsolicited': False,
 'schemaVersion': 'v1',
 'gatewaySerialNumber': 'PMM-QA0002',
 'movementMeasurement': {'initialAcceleration': {'x': 48,
   'y': -60,
   'z': -1005}},
 'packetNumberGateway': 84197,
 'pressureMeasurement': 1017,
 'humidityMeasurements': [{'value': 32.75, 'sensorIndex': 0}],
 'packetNumberEversensor': 24,
 'temperatureMeasurements': [{'value': 295.25, 'sensorIndex': 0},
  {'value': 240, 'sensorIndex': 1}]}

### Pull Last 24 Hours of Reported Readings for Eversensor

In [9]:
start_time = last_reading["timestamp"] - 24*60*60
end_time = last_reading["timestamp"]

df = api.get_eversensor_readings(
    sensor_mac_address,
    start_time=start_time,
    end_time=end_time,
    output_format="pandas"
)

In [10]:
print(df.shape)
df.head()

(0, 0)


""


### Identify Eversensor Movement Events

If there are detected movement events in your retrieved eversensor data, you will see a `movementMeasurement_movement` column in the results DataFrame. We'll pull out any movement events present in the sensor readings.

In [11]:
movement_events = []

if "movementMeasurement_movement" in df.columns:
    movement_detected = df["movementMeasurement_movement"].fillna(False)

    for _, row in df[movement_detected].iterrows():
        movement_events.append({
            "macAddress": row["macAddress"],
            "timestamp" : row["timestamp"],
            "readingDate" : row["readingDate"],
        })

df_movements = pd.DataFrame(movement_events)

In [12]:
df_movements.head()

""


## Visualize Eversensor Readings

In [13]:
CHART_HEIGHT = 300
CHART_WIDTH = 700

In [14]:
pressure = alt.Chart(df).mark_circle().encode(
    alt.X("readingDate:T"),
    alt.Y("pressureMeasurement:Q")
)

movements = alt.Chart(df_movements).mark_rule().encode(
    alt.X("readingDate:T")
)

alt.layer(pressure, movements).properties(height=CHART_HEIGHT, width=CHART_WIDTH)

/root/.cache/pypoetry/virtualenvs/everactive-envplus-9CdbqzWc-py3.9/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

In [15]:
rssi = alt.Chart(df).mark_circle().encode(
    alt.X("readingDate:T"),
    alt.Y("rssiUplink:Q")
)

movements = alt.Chart(df_movements).mark_rule().encode(
    alt.X("readingDate:T")
)

alt.layer(rssi, movements).properties(height=CHART_HEIGHT, width=CHART_WIDTH)

alt.LayerChart(...)

## Join the Everactive Community